In [294]:
import numpy as np
import pandas as pd
from math import log2 as log
eps = np.finfo(float).eps

In [295]:
# 1: Nhập dữ liệu vào một danh sách
matrix = [
    ["D1", "Sunny", "High", "Weak", "No"],
    ["D2", "Sunny", "High", "Strong", "No"],
    ["D3", "Overcast", "High", "Weak", "Yes"],
    ["D4", "Rain", "High", "Weak", "Yes"],
    ["D5", "Rain", "Normal", "Weak", "Yes"],
    ["D6", "Rain", "Normal", "Strong", "No"],
    ["D7", "Overcast", "Normal", "Strong", "Yes"],
    ["D8", "Sunny", "High", "Weak", "No"],
    ["D9", "Sunny", "Normal", "Weak", "Yes"],
    ["D10", "Rain", "Normal", "Weak", "Yes"],
    ["D11", "Sunny", "Normal", "Strong", "Yes"],
    ["D12", "Overcast", "High", "Strong", "Yes"],
    ["D13", "Overcast", "Normal", "Weak", "Yes"],
    ["D14", "Rain", "High", "Strong", "No"]
]
data = pd.DataFrame(matrix, columns = ["Day","Outlook", "Humidity", "Wind", "PlayTennis"])
outlook = np.array(data[["Outlook"]]).reshape(-1)
humidity = np.array(data[["Humidity"]]).reshape(-1)
wind = np.array(data[["Wind"]]).reshape(-1)
play = np.array(data[["PlayTennis"]]).reshape(-1)

dataset ={'outlook':outlook,'humidity':humidity,'wind':wind,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','humidity','wind','play'])
df

,outlook,humidity,wind,play
0,Sunny,High,Weak,No
1,Sunny,High,Strong,No
2,Overcast,High,Weak,Yes
3,Rain,High,Weak,Yes
4,Rain,Normal,Weak,Yes
5,Rain,Normal,Strong,No
6,Overcast,Normal,Strong,Yes
7,Sunny,High,Weak,No
8,Sunny,Normal,Weak,Yes
9,Rain,Normal,Weak,Yes


In [297]:
def find_entropy(df):
    Class = df.keys()[-1]
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

def find_entropy_attribute(df,attribute):
    '''tính entropy của một thuộc tính trong DataFrame df. Tính entropy cho mỗi giá trị duy nhất của thuộc tính và trả về tổng entropy'''
    Class = df.keys()[-1]
    target_variables = df[Class].unique()
    variables = df[attribute].unique()  
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

def find_winner(df):
    '''thuộc tính có entropy cao nhất, được chọn làm nút gốc cho cây quyết định'''
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)] 

def get_subtable(df, node,value):
    '''trích xuất một DataFrame con từ DataFrame df với val cụ thể '''
    return df[df[node] == value].reset_index(drop=True)

def buildTree(df,tree=None):
    '''xây dựng cây quyết định từ DataFrame'''
    Class = df.keys()[-1]   
    node = find_winner(df)
    attValue = np.unique(df[node]) 
    if tree is None:                    
        tree={}
        tree[node] = {}
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['play'],return_counts=True)                        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable)           
    return tree

tree= buildTree(df)
import pprint
print("CÂY QUYẾT ĐỊNH")
pprint.pprint(tree)

CÂY QUYẾT ĐỊNH
{'outlook': {'Overcast': 'Yes',
             'Rain': {'wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}
